In [1]:
from src.data_structures import Instance
from src.data_structures.features import *
import torch
from torch import nn
import os
from pathlib import Path

In [85]:
#Training data (Instancias ya solucionadas)
max_size = 1000
training_data_folder = Path("training_data")
instances = map(Instance.from_file,training_data_folder.iterdir())
instance = next(instances.__iter__())
expand_vector = lambda x: torch.cat([x,torch.zeros(max_size-instance.n_items)])
get_y = lambda instance: expand_vector(torch.tensor(instance.optimal_solution))

In [86]:
y = get_y(instance)

In [78]:
#Features
#Debo generar un vector x por cada feature, y cada vector x debe expandirse,
#y luego concatenar todos los vectores x expandidos por lo que la dimension de entrada
#sera #features*n_items.

#Esto se hace por cada instancia

features: list[ItemBatchFeature] = [ProfitOverBudget,LowerCostOverBudget]
evaluated_features = []
for feature in features:
    x_feature = torch.tensor(feature.batch_evaluate(instance))
    x_feature = expand_vector(x_feature)
    evaluated_features.append(x_feature)
    x = torch.cat(evaluated_features)

2000


In [ ]:
class Net(nn.Module):
    def __init__(self, entrada,salida):
        super(Net, self).__init__()

        hidden_size = 100

        self.many = nn.Sequential(
            nn.Linear(entrada, hidden_size),
            nn.Linear(hidden_size,salida),
            nn.Sigmoid()
        )

    def forward(self, x):   
        x = self.many(x)
        return x

In [79]:
net = Net(len(features)*max_size,max_size)

In [3]:
#Training
for instance in instances:
    total_loss = 0
    optimizer.zero_grad()
        batch_output = net(batch_x)
        batch_loss = criterion(batch_output, batch_y)
        batch_loss.backward()
        optimizer.step()
        total_loss += batch_loss.item()    
    sys.stdout.write(f'\rEpoch {epoch}, Loss {total_loss / len(dataloader)}')
    sys.stdout.flush()

